In [1]:
import os
from itertools import islice

import numpy as np
import pandas as pd

from skimage.external import tifffile
from scipy.ndimage import gaussian_filter

import ipywidgets as widgets

from scipy import ndimage
from PIL import Image, ImageDraw

import numpy.ma as ma

from skimage.measure import label, regionprops
import matplotlib.pyplot as plt

In [2]:
bigDir = r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\DATA'

In [3]:
myFile= r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\sel_cells_20_50_10_newctrl.csv'

In [4]:
myBigData=pd.read_csv(myFile)

In [5]:
myBigData.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,file,original cell name,Amy_mod,movie,x,y,...,random_10perc_ch1,hetChrom_10perc_ch2,random_10perc_ch2,hetChrom_10perc_ch3,random_10perc_ch3,exp_folder,hetChrom_newctl_volume,hetChrom_newctl_ch1,hetChrom_newctl_ch2,hetChrom_newctl_ch3
0,0,0,0,0,20190625_cell-00.czi,7LE1(B),1130/10,190619-cdc6-dhb-pcna002xy09,172,956,...,281528813.0,174067475.0,131640512.0,1.352231e+09,1.043517e+09,20190625-MSL rename,280370,5.739116e+09,3.726297e+09,9.123801e+09
1,1,1,1,1,20190625_cell-01.czi,7L-E1(T),0,190619-cdc6-dhb-pcna002xy09,158,906,...,266516067.0,162411790.0,128299406.0,1.353023e+09,1.065184e+09,20190625-MSL rename,250606,5.403750e+09,3.628343e+09,8.997863e+09
2,2,2,2,2,20190625_cell-02.czi,7L-E2(B),0,190619-cdc6-dhb-pcna002xy09,832,252,...,318905411.0,194622718.0,148176740.0,1.366670e+09,1.118365e+09,20190625-MSL rename,189925,6.729417e+09,4.442309e+09,1.076495e+10
3,3,3,3,3,20190625_cell-03.czi,7L-E2(T),0,190619-cdc6-dhb-pcna002xy09,888,160,...,285348983.0,195376713.0,139563277.0,1.298882e+09,1.038559e+09,20190625-MSL rename,220076,5.459073e+09,3.733122e+09,9.257708e+09
4,4,4,4,4,20190625_cell-05.czi,7L-G2,0,190619-cdc6-dhb-pcna002xy09,172,324,...,698512214.0,248463390.0,183210463.0,2.583220e+09,2.158853e+09,20190625-MSL rename,436430,8.007695e+09,4.024894e+09,1.674243e+10


In [ ]:
i=0

# Progress Bar
progBar=widgets.IntProgress(
    value=i,
    min=0,
    max=len(myBigData),
    step=1,
    description='Progress:',
    orientation='horizontal'
)
display(progBar)

for i, cell in myBigData.iterrows():
    
    # open tiff image of a cell
    myImage=imread(myCell.path)    
    myChannel = myImage[:, 0, :, :] #1st channel is hetChrom (as stained by HP1B or H3K9Me3)
    
    # open nucleus mask
    nucleusMaskPath=myCell.path
    nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
    nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
    nucleusMask=imread(nucleusMaskPath)
    nucleusMask=nucleusMask.astype(bool)

    

    ## finding nuc boundary

    z_axis=np.argmax(nucMask,axis=0)
    t=label(z_axis>0)
    props=regionprops(t)
    prop=props[0].bbox

    x_min = prop[0]
    x_max = prop[2]
    y_min = prop[1]
    y_max = prop[3]

    y_axis=np.argmax(nucMask,axis=2)
    t=label(y_axis>0)
    props=regionprops(t)
    prop=props[0].bbox

    z_min = prop[0]
    z_max = prop[2]


    ## finding center of the nucleus
    
    
    
    ## masking heterochromatin with the box-mask

    nucMask_with_cube_copy=nucMask_with_cube.copy()
    nucMask_with_cube_copy=nucMask_with_cube_copy.astype(int)
    het_copy=het.copy()
    het_copy=het_copy.astype(int)
    het_box_mask=het_copy[z_min:z_max, x_min:x_max, y_min:y_max]
    het_mask_with_box=ma.zeros(het_copy.shape)
    het_mask_with_box[z_min:z_max, x_min:x_max, y_min:y_max]=het_box_mask

    
        
    # calculating signals
    df = pd.DataFrame(columns = ['HetChrom_inner_Volume','HetChrom_rad_inner_ch1','HetChrom_rad_inner_ch2','HetChrom_rad_inner_ch3',
                                'HetChrom_outer_Volume','HetChrom_rad_outer_ch1','HetChrom_rad_outer_ch2','HetChrom_rad_outer_ch3'])

    df.Slice = range(myChannel.shape[0])
    df.HetChrom_inner_Volume =  myHetMaskInner.sum(-1).sum(-1)
    df.HetChrom_outer_Volume =  myHetMaskOuter.sum(-1).sum(-1)

    #df.random_Volume =  scrMask.sum(-1).sum(-1)

    for ch in range(3):

        # inner
        myChannel = np.copy(myImage[:, ch, :, :])
        myChannel[myHetMaskInner] = 0
        signal = myChannel.sum(-1).sum(-1)
        exec(f'df.HetChrom_rad_inner_ch{ch+1}=signal')

        # outer
        myChannel = np.copy(myImage[:, ch, :, :])
        myChannel[myHetMaskOuter] = 0
        signal = myChannel.sum(-1).sum(-1)
        exec(f'df.HetChrom_rad_outer_ch{ch+1}=signal')



    # save signals
    '''
    dfSavePath=myCell.path
    dfSavePath=dfSavePath.replace('data_tiff','results')
    dfSavePath=dfSavePath.replace('.tif',f'_hetChrom_newctl.csv')
    df.to_csv(dfSavePath)
    '''

    # add info to cell info file
    cellData.loc[i,f'HetChrom_inner_Volume']=np.sum(df.HetChrom_inner_Volume)
    cellData.loc[i,f'HetChrom_outer_Volume']=np.sum(df.HetChrom_outer_Volume)


    for ch in range(3):
        exec(f"cellData.loc[i,'HetChrom_rad_outer_ch{ch+1}']=np.sum(df.HetChrom_rad_outer_ch{ch+1})") 
        exec(f"cellData.loc[i,'HetChrom_rad_inner_ch{ch+1}']=np.sum(df.HetChrom_rad_inner_ch{ch+1})") 


    break
    progBar.value=progBar.value+1
    